In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import copy

from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

#Dataset

In [ ]:
#22/08/2019 -> 26/09/2019
output_true_A = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/Dataset/output_prediction/output_true_A.csv')
output_true_B = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/Dataset/output_prediction/output_true_B.csv')
output_true_C = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/Dataset/output_prediction/output_true_C.csv')
output_true_D = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/Dataset/output_prediction/output_true_D.csv')

output_pred_A = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/Dataset/output_prediction/output_pred_A.csv')
output_pred_B = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/Dataset/output_prediction/output_pred_B.csv')
output_pred_C = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/Dataset/output_prediction/output_pred_C.csv')
output_pred_D = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/Dataset/output_prediction/output_pred_D.csv')

output_pred_all_A = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/Dataset/output_prediction/output_pred_all_A.csv')
output_pred_all_B = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/Dataset/output_prediction/output_pred_all_B.csv')
output_pred_all_C = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/Dataset/output_prediction/output_pred_all_C.csv')
output_pred_all_D = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/Dataset/output_prediction/output_pred_all_D.csv')


In [ ]:
len(output_pred_A),len(output_pred_B),len(output_pred_C),len(output_pred_D)

In [ ]:
252+1908+4284+169992

In [ ]:
true = output_true_A.append([output_true_B,output_true_C,output_true_D])
pred_g = output_pred_A.append([output_pred_B,output_pred_C,output_pred_D])
pred_all = output_pred_all_A.append([output_pred_all_B,output_pred_all_C,output_pred_all_D])

#Over_all

##Estimates

In [ ]:
output_true_A = true.replace('Monday', '1Monday')
output_true_A = output_true_A.replace('Tuesday', '2Tuesday')
output_true_A = output_true_A.replace('Wednesday', '3Wednesday')
output_true_A = output_true_A.replace('Thursday', '4Thursday')
output_true_A = output_true_A.replace('Friday', '5Friday')
output_true_A = output_true_A.replace('Saturday', '6Saturday')
output_true_A = output_true_A.replace('Sunday', '7Sunday')

dayA = output_true_A['start_day'].copy()

estimates = output_true_A.copy()
estimates = estimates.iloc[0:0]
estimateA = estimates.drop(columns=['start_date', 'start_day','grid_ID'])

estimate_a = pd.DataFrame()

for i in range(24) :
  #true - predict
  estimateA[str(i)] = true[str(i)] - pred_g[str(i)]

over_estimate_a = estimateA.where(estimateA >= 0 )
under_estimate_a = estimateA.where(estimateA <= 0)

over_a = pd.merge(dayA, over_estimate_a, left_index=True, right_index=True)
under_a = pd.merge(dayA, under_estimate_a, left_index=True, right_index=True)

over_estimate_a = over_a.groupby(by=["start_day"]).mean()
under_estimate_a = under_a.groupby(by=["start_day"]).mean()
estimate_a['over_estimate'] = over_estimate_a.mean(axis=1)
estimate_a['under_estimate'] = under_estimate_a.mean(axis=1)
estimate_a['over_estimate_std'] = over_estimate_a.std(axis = 1)
estimate_a['under_estimate_std'] = under_estimate_a.std(axis = 1)

estimate_a = estimate_a.reset_index()

# estimate_a

In [ ]:
estimates = true.copy()
estimates = estimates.iloc[0:0]

estimatenA = estimates.drop(columns=['start_date', 'start_day','grid_ID'])

estimate_na = pd.DataFrame()
for i in range(24) :
  #true - predict
  estimatenA[str(i)] = true[str(i)] - pred_all[str(i)]

over_estimate_na = estimatenA.where(estimatenA >= 0 )
under_estimate_na = estimatenA.where(estimatenA <= 0)

over_na = pd.merge(dayA, over_estimate_na, left_index=True, right_index=True)
under_na = pd.merge(dayA, under_estimate_na, left_index=True, right_index=True)

over_estimate_na = over_na.groupby(by=["start_day"]).mean()
under_estimate_na = under_na.groupby(by=["start_day"]).mean()
estimate_na['over_estimate'] = over_estimate_na.mean(axis=1)
estimate_na['under_estimate'] = under_estimate_na.mean(axis=1)
estimate_na['over_estimate_std'] = over_estimate_a.std(axis = 1)
estimate_na['under_estimate_std'] = under_estimate_a.std(axis = 1)
estimate_na = estimate_na.reset_index()
# estimate_na

In [ ]:
fig, axs = plt.subplots(figsize=(12,7))


# fig.suptitle('Grouping-based training',fontsize=15,verticalalignment='bottom')
labels = ["over_estimate_none", "under_estimate_none","over_estimate_group", "under_estimate_group"]

os = estimate_a.over_estimate_std.to_list()
om = estimate_a.over_estimate.tolist()
us = estimate_a.under_estimate_std.to_list()
um = estimate_a.under_estimate.tolist()

nos = estimate_na.over_estimate_std.to_list()
nom = estimate_na.over_estimate.tolist()
nus = estimate_na.under_estimate_std.to_list()
num = estimate_na.under_estimate.tolist()

d = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']

axs.errorbar(d,nom,yerr=nos,marker='o',
         mec='black', ms=2, mew=2, capsize=5,label=labels[0])
axs.errorbar(d,num,yerr=nus,marker='s', mfc='red',
         mec='red', ms=2, mew=2, capsize=5,label=labels[1])

axs.errorbar(d,om,yerr=os,marker='o',
         mec='black', ms=2, mew=2, capsize=5,label=labels[2])
axs.errorbar(d,um,yerr=us,marker='s', mfc='red',
         mec='red', ms=2, mew=2, capsize=5,label=labels[3])

# axs.plot(estimate_na.index,estimate_na.over_estimate,label=labels[0])
# axs.plot(estimate_na.index,estimate_na.under_estimate,label=labels[1])
# axs.plot(estimate_a.index,estimate_a.over_estimate,label=labels[2])
# axs.plot(estimate_a.index,estimate_a.under_estimate,label=labels[3])

axs.set_title('Overall results Estimates')
axs.legend(loc="upper left")
axs.set(xlabel='day of the week', ylabel='estimate')

plt.tight_layout()

plt.show();

In [ ]:
fig, axs = plt.subplots(figsize=(12,7))


# fig.suptitle('Grouping-based training',fontsize=15,verticalalignment='bottom')
# labels = ["over_estimate_none", "under_estimate_none","over_estimate_group", "under_estimate_group"]

os = estimate_a.over_estimate_std.to_list()
om = estimate_a.over_estimate.tolist()
us = estimate_a.under_estimate_std.to_list()
um = estimate_a.under_estimate.tolist()

nos = estimate_na.over_estimate_std.to_list()
nom = estimate_na.over_estimate.tolist()
nus = estimate_na.under_estimate_std.to_list()
num = estimate_na.under_estimate.tolist()

d = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']

axs.errorbar(d,nom,yerr=nos,marker='o',
         mec='black', ms=2, mew=2, capsize=5)
axs.errorbar(d,num,yerr=nus,marker='s', mfc='red',
         mec='red', ms=2, mew=2, capsize=5)



# axs.plot(estimate_na.index,estimate_na.over_estimate,label=labels[0])
# axs.plot(estimate_na.index,estimate_na.under_estimate,label=labels[1])
# axs.plot(estimate_a.index,estimate_a.over_estimate,label=labels[2])
# axs.plot(estimate_a.index,estimate_a.under_estimate,label=labels[3])

axs.set_title('Overall results Estimates(Non-Group)')
# axs.legend(loc="upper left")
axs.set(xlabel='day of the week', ylabel='estimate')

plt.tight_layout()

plt.show();

##Zero_acc

In [ ]:
output_true_A = true.replace('Monday', '1Monday')
output_true_A = output_true_A.replace('Tuesday', '2Tuesday')
output_true_A = output_true_A.replace('Wednesday', '3Wednesday')
output_true_A = output_true_A.replace('Thursday', '4Thursday')
output_true_A = output_true_A.replace('Friday', '5Friday')
output_true_A = output_true_A.replace('Saturday', '6Saturday')
output_true_A = output_true_A.replace('Sunday', '7Sunday')

dayA = output_true_A['start_day'].copy()

zeror_a = pd.DataFrame()
zeror_ = pd.DataFrame()
zero_ = pd.DataFrame()
zero_all = pd.DataFrame()
output = pd.DataFrame()
output_all = pd.DataFrame()

out = pd.DataFrame()

for i in range(24) :
  zeror_a[str(i)] = np.where((true[str(i)] == 0)
                     ,0, np.nan)

a = pd.merge(dayA, zeror_a, left_index=True, right_index=True)
out = a.groupby(by=["start_day"]).count().reset_index()
day = out.iloc[:, 0].reset_index()
out = out.drop(['start_day'], axis=1)
out = out.sum(axis=1).to_list()
sss = pd.DataFrame(out, columns=['sums'])

results_t = pd.merge(day, sss, left_index=True, right_index=True)
results_t = results_t.drop(['index'], axis=1)
results_t

In [ ]:
for i in range(24) :
  zero_a[str(i)] = np.where((true[str(i)] == pred_g[str(i)]) & (pred_g[str(i)] == 0)
                     ,0, np.nan)

b = pd.merge(dayA, zero_a, left_index=True, right_index=True)
outs = b.groupby(by=["start_day"]).count().reset_index()
day = outs.iloc[:, 0].reset_index()
outs = outs.drop(['start_day'], axis=1)
outs = outs.sum(axis=1).to_list()
www = pd.DataFrame(outs, columns=['sum'])

results = pd.merge(day, www, left_index=True, right_index=True)
results = results.drop(['index'], axis=1)

output['Zero_Acc'] = (results['sum']*100) / results_t['sums']
output = output.fillna(0)
output

In [ ]:
for i in range(24) :
  zero_all[str(i)] = np.where((true[str(i)] == pred_all[str(i)]) & (pred_all[str(i)] == 0)
                     ,0, np.nan)

c = pd.merge(dayA, zero_all, left_index=True, right_index=True)
outss = c.groupby(by=["start_day"]).count().reset_index()
days = outss.iloc[:, 0].reset_index()
outss = outss.drop(['start_day'], axis=1)
outss = outss.sum(axis=1).to_list()
wwws = pd.DataFrame(outss, columns=['sum'])

resultss = pd.merge(day, wwws, left_index=True, right_index=True)
# resultss = results.drop(['index'], axis=1)

output_all['Zero_Acc_all'] = (resultss['sum']*100) / results_t['sums']
output_all = output_all.fillna(0)
output_all

last_output = pd.merge(day, output, left_index=True, right_index=True)
last_output = pd.merge(last_output, output_all, left_index=True, right_index=True)
last_output = last_output.drop(['Zero_Acc_y','index'], axis=1)
last_output

In [ ]:
fig, axs = plt.subplots(figsize=(12,7))

labels = ["Zero_acc_non","Zero_acc_group"]
# fig.suptitle('Zero Accuracy',fontsize=15,verticalalignment='bottom')

axs.plot(last_output.start_day,last_output.Zero_Acc_all,label=labels[0])
axs.plot(last_output.start_day,last_output.Zero_Acc_x,label=labels[1])
axs.set_title('Overall results Zero Accuracy')
axs.legend(loc="upper right")

axs.set(xlabel='hour', ylabel='Accuracy(%)')

# for ax in axs.flat:
#     ax.set(xlabel='hour', ylabel='Accuracy(%)')

plt.tight_layout()

plt.show();

#Grouping-based training

In [ ]:
output_true_A = output_true_A.replace('Monday', '1Monday')
output_true_A = output_true_A.replace('Tuesday', '2Tuesday')
output_true_A = output_true_A.replace('Wednesday', '3Wednesday')
output_true_A = output_true_A.replace('Thursday', '4Thursday')
output_true_A = output_true_A.replace('Friday', '5Friday')
output_true_A = output_true_A.replace('Saturday', '6Saturday')
output_true_A = output_true_A.replace('Sunday', '7Sunday')

output_true_B = output_true_B.replace('Monday', '1Monday')
output_true_B = output_true_B.replace('Tuesday', '2Tuesday')
output_true_B = output_true_B.replace('Wednesday', '3Wednesday')
output_true_B = output_true_B.replace('Thursday', '4Thursday')
output_true_B = output_true_B.replace('Friday', '5Friday')
output_true_B = output_true_B.replace('Saturday', '6Saturday')
output_true_B = output_true_B.replace('Sunday', '7Sunday')

output_true_C = output_true_C.replace('Monday', '1Monday')
output_true_C = output_true_C.replace('Tuesday', '2Tuesday')
output_true_C = output_true_C.replace('Wednesday', '3Wednesday')
output_true_C = output_true_C.replace('Thursday', '4Thursday')
output_true_C = output_true_C.replace('Friday', '5Friday')
output_true_C = output_true_C.replace('Saturday', '6Saturday')
output_true_C = output_true_C.replace('Sunday', '7Sunday')

output_true_D = output_true_D.replace('Monday', '1Monday')
output_true_D = output_true_D.replace('Tuesday', '2Tuesday')
output_true_D = output_true_D.replace('Wednesday', '3Wednesday')
output_true_D = output_true_D.replace('Thursday', '4Thursday')
output_true_D = output_true_D.replace('Friday', '5Friday')
output_true_D = output_true_D.replace('Saturday', '6Saturday')
output_true_D = output_true_D.replace('Sunday', '7Sunday')


dayA = output_true_A['start_day'].copy()
dayB = output_true_B['start_day'].copy()
dayC = output_true_C['start_day'].copy()
dayD = output_true_D['start_day'].copy()

In [ ]:
estimates = output_true_A.copy()
estimates = estimates.iloc[0:0]

estimateA = estimates.drop(columns=['start_date', 'start_day','grid_ID'])
estimateB = estimateA.copy()
estimateC = estimateA.copy()
estimateD = estimateA.copy()

estimate_a = pd.DataFrame()
estimate_b = pd.DataFrame()
estimate_c = pd.DataFrame()
estimate_d = pd.DataFrame()

In [ ]:
for i in range(24) :
  #true - predict
  estimateA[str(i)] = output_true_A[str(i)] - output_pred_A[str(i)]
  estimateB[str(i)] = output_true_B[str(i)] - output_pred_B[str(i)]
  estimateC[str(i)] = output_true_C[str(i)] - output_pred_C[str(i)]
  estimateD[str(i)] = output_true_D[str(i)] - output_pred_D[str(i)]

In [ ]:
#over_estimate / true - predict = +
#under_estimate / true - predict = -
#if true - predict = 0 -> add 0 both
#---------------------------A-------------------------------------------#
over_estimate_a = estimateA.where(estimateA >= 0 )
under_estimate_a = estimateA.where(estimateA <= 0)

over_a = pd.merge(dayA, over_estimate_a, left_index=True, right_index=True)
under_a = pd.merge(dayA, under_estimate_a, left_index=True, right_index=True)

over_estimate_a = over_a.groupby(by=["start_day"]).mean()
under_estimate_a = under_a.groupby(by=["start_day"]).mean()
estimate_a['over_estimate'] = over_estimate_a.mean(axis=1)
estimate_a['under_estimate'] = under_estimate_a.mean(axis=1)
estimate_a = estimate_a.reset_index()

#---------------------------B-------------------------------------------#
over_estimate_b = estimateB.where(estimateB >= 0 )
under_estimate_b = estimateB.where(estimateB <= 0)

over_b = pd.merge(dayB, over_estimate_b, left_index=True, right_index=True)
under_b = pd.merge(dayB, under_estimate_b, left_index=True, right_index=True)

over_estimate_b = over_b.groupby(by=["start_day"]).mean()
under_estimate_b = under_b.groupby(by=["start_day"]).mean()
estimate_b['over_estimate'] = over_estimate_b.mean(axis = 1)
estimate_b['under_estimate'] = under_estimate_b.mean(axis = 1)
estimate_b = estimate_b.reset_index()

#---------------------------C-------------------------------------------#
over_estimate_c = estimateC.where(estimateC >= 0 )
under_estimate_c = estimateC.where(estimateC <= 0)

over_c = pd.merge(dayC, over_estimate_c, left_index=True, right_index=True)
under_c = pd.merge(dayC, under_estimate_c, left_index=True, right_index=True)

over_estimate_c = over_c.groupby(by=["start_day"]).mean()
under_estimate_c = under_c.groupby(by=["start_day"]).mean()
estimate_c['over_estimate'] = over_estimate_c.mean(axis = 1, skipna = True)
estimate_c['under_estimate'] = under_estimate_c.mean(axis = 1, skipna = True)
estimate_c = estimate_c.reset_index()

#---------------------------D-------------------------------------------#
over_estimate_d = estimateD.where(estimateD >= 0 )
under_estimate_d = estimateD.where(estimateD <= 0)

over_d = pd.merge(dayD, over_estimate_d, left_index=True, right_index=True)
under_d = pd.merge(dayD, under_estimate_d, left_index=True, right_index=True)

over_estimate_d = over_d.groupby(by=["start_day"]).mean()
under_estimate_d = under_d.groupby(by=["start_day"]).mean()
estimate_d['over_estimate'] = over_estimate_d.mean(axis = 1, skipna = True)
estimate_d['under_estimate'] = under_estimate_d.mean(axis = 1, skipna = True)
estimate_d = estimate_d.reset_index()

In [ ]:
estimate_a

##ploting

In [ ]:
fig, axs = plt.subplots(2, 2,figsize=(18,10))

labels = ["over_estimate", "under_estimate"]
fig.suptitle('Grouping-based training',fontsize=15,verticalalignment='bottom')

axs[0, 0].plot(estimate_a.start_day,estimate_a.over_estimate,label=labels[0])
axs[0, 0].plot(estimate_a.start_day,estimate_a.under_estimate,label=labels[1])
axs[0, 0].set_title('Group A')
axs[0, 0].legend(loc="upper left")

axs[0, 1].plot(estimate_b.start_day,estimate_b.over_estimate,label=labels[0])
axs[0, 1].plot(estimate_b.start_day,estimate_b.under_estimate,label=labels[1])
axs[0, 1].set_title('Group B')
axs[0, 1].legend(loc="upper left")

axs[1, 0].plot(estimate_c.start_day,estimate_c.over_estimate,label=labels[0])
axs[1, 0].plot(estimate_c.start_day,estimate_c.under_estimate,label=labels[1])
axs[1, 0].set_title('Group C')
axs[1, 0].legend(loc="upper left")

axs[1, 1].plot(estimate_d.start_day,estimate_d.over_estimate,label=labels[0])
axs[1, 1].plot(estimate_d.start_day,estimate_d.under_estimate,label=labels[1])
axs[1, 1].set_title('Group D')
axs[1, 1].legend(loc="upper left")


for ax in axs.flat:
    ax.set(xlabel='hour', ylabel='average estimate')

plt.tight_layout()

plt.show();

#None Grouping-based training

In [ ]:
estimates = output_true_A.copy()
estimates = estimates.iloc[0:0]

estimatenA = estimates.drop(columns=['start_date', 'start_day','grid_ID'])
estimatenB = estimatenA.copy()
estimatenC = estimatenA.copy()
estimatenD = estimatenA.copy()

estimate_na = pd.DataFrame()
estimate_nb = pd.DataFrame()
estimate_nc = pd.DataFrame()
estimate_nd = pd.DataFrame()

In [ ]:
for i in range(24) :
  #true - predict
  estimatenA[str(i)] = output_true_A[str(i)] - output_pred_all_A[str(i)]
  estimatenB[str(i)] = output_true_B[str(i)] - output_pred_all_B[str(i)]
  estimatenC[str(i)] = output_true_C[str(i)] - output_pred_all_C[str(i)]
  estimatenD[str(i)] = output_true_D[str(i)] - output_pred_all_D[str(i)]

#over_estimate / true - predict = +
#under_estimate / true - predict = -
#if true - predict = 0 -> add 0 both
#---------------------------A-------------------------------------------#
over_estimate_na = estimatenA.where(estimatenA >= 0 )
under_estimate_na = estimatenA.where(estimatenA <= 0)

over_na = pd.merge(dayA, over_estimate_na, left_index=True, right_index=True)
under_na = pd.merge(dayA, under_estimate_na, left_index=True, right_index=True)

over_estimate_na = over_na.groupby(by=["start_day"]).mean()
under_estimate_na = under_na.groupby(by=["start_day"]).mean()
estimate_na['over_estimate'] = over_estimate_na.mean(axis=1)
estimate_na['under_estimate'] = under_estimate_na.mean(axis=1)
estimate_na = estimate_na.reset_index()

#---------------------------B-------------------------------------------#
over_estimate_nb = estimatenB.where(estimatenB >= 0 )
under_estimate_nb = estimatenB.where(estimatenB <= 0)

over_nb = pd.merge(dayB, over_estimate_nb, left_index=True, right_index=True)
under_nb = pd.merge(dayB, under_estimate_nb, left_index=True, right_index=True)

over_estimate_nb = over_nb.groupby(by=["start_day"]).mean()
under_estimate_nb = under_nb.groupby(by=["start_day"]).mean()
estimate_nb['over_estimate'] = over_estimate_nb.mean(axis = 1)
estimate_nb['under_estimate'] = under_estimate_nb.mean(axis = 1)
estimate_nb = estimate_nb.reset_index()

#---------------------------C-------------------------------------------#
over_estimate_nc = estimatenC.where(estimatenC >= 0 )
under_estimate_nc = estimatenC.where(estimatenC <= 0)

over_nc = pd.merge(dayC, over_estimate_nc, left_index=True, right_index=True)
under_nc = pd.merge(dayC, under_estimate_nc, left_index=True, right_index=True)

over_estimate_nc = over_nc.groupby(by=["start_day"]).mean()
under_estimate_nc = under_nc.groupby(by=["start_day"]).mean()
estimate_nc['over_estimate'] = over_estimate_nc.mean(axis = 1, skipna = True)
estimate_nc['under_estimate'] = under_estimate_nc.mean(axis = 1, skipna = True)
estimate_nc = estimate_nc.reset_index()

#---------------------------D-------------------------------------------#
over_estimate_nd = estimatenD.where(estimatenD >= 0 )
under_estimate_nd = estimatenD.where(estimatenD <= 0)

over_nd = pd.merge(dayD, over_estimate_nd, left_index=True, right_index=True)
under_nd = pd.merge(dayD, under_estimate_nd, left_index=True, right_index=True)

over_estimate_nd = over_nd.groupby(by=["start_day"]).mean()
under_estimate_nd = under_nd.groupby(by=["start_day"]).mean()
estimate_nd['over_estimate'] = over_estimate_nd.mean(axis = 1, skipna = True)
estimate_nd['under_estimate'] = under_estimate_nd.mean(axis = 1, skipna = True)
estimate_nd = estimate_nd.reset_index()

##ploting

In [ ]:
fig, axs = plt.subplots(2, 2,figsize=(18,10))

labels = ["over_estimate", "under_estimate"]
fig.suptitle('None Grouping training',fontsize=15,verticalalignment='bottom')

axs[0, 0].plot(estimate_na.start_day,estimate_na.over_estimate,label=labels[0])
axs[0, 0].plot(estimate_na.start_day,estimate_na.under_estimate,label=labels[1])
axs[0, 0].set_title('Group A')
axs[0, 0].legend(loc="upper left")

axs[0, 1].plot(estimate_nb.start_day,estimate_nb.over_estimate,label=labels[0])
axs[0, 1].plot(estimate_nb.start_day,estimate_nb.under_estimate,label=labels[1])
axs[0, 1].set_title('Group B')
axs[0, 1].legend(loc="upper left")

axs[1, 0].plot(estimate_nc.start_day,estimate_nc.over_estimate,label=labels[0])
axs[1, 0].plot(estimate_nc.start_day,estimate_nc.under_estimate,label=labels[1])
axs[1, 0].set_title('Group C')
axs[1, 0].legend(loc="upper left")

axs[1, 1].plot(estimate_nd.start_day,estimate_nd.over_estimate,label=labels[0])
axs[1, 1].plot(estimate_nd.start_day,estimate_nd.under_estimate,label=labels[1])
axs[1, 1].set_title('Group D')
axs[1, 1].legend(loc="upper left")


for ax in axs.flat:
    ax.set(xlabel='hour', ylabel='average estimate')

plt.tight_layout()

plt.show();

#measure the performance 

##Comparing plot

In [ ]:
fig, axs = plt.subplots(2, 2,figsize=(18,10))

labels = ["oen", "uen","oeg", "ueg"]
fig.suptitle('Grouping-based training Vs None Grouping training',fontsize=15,verticalalignment='bottom')

axs[0, 0].plot(estimate_na.start_day,estimate_na.over_estimate,label=labels[0])
axs[0, 0].plot(estimate_na.start_day,estimate_na.under_estimate,label=labels[1])
axs[0, 0].plot(estimate_a.start_day,estimate_a.over_estimate,label=labels[2])
axs[0, 0].plot(estimate_a.start_day,estimate_a.under_estimate,label=labels[3])
axs[0, 0].set_title('Group A')
axs[0, 0].legend(loc="upper left")

axs[0, 1].plot(estimate_nb.start_day,estimate_nb.over_estimate,label=labels[0])
axs[0, 1].plot(estimate_nb.start_day,estimate_nb.under_estimate,label=labels[1])
axs[0, 1].plot(estimate_b.start_day,estimate_b.over_estimate,label=labels[2])
axs[0, 1].plot(estimate_b.start_day,estimate_b.under_estimate,label=labels[3])
axs[0, 1].set_title('Group B')
axs[0, 1].legend(loc="upper left")

axs[1, 0].plot(estimate_nc.start_day,estimate_nc.over_estimate,label=labels[0])
axs[1, 0].plot(estimate_nc.start_day,estimate_nc.under_estimate,label=labels[1])
axs[1, 0].plot(estimate_c.start_day,estimate_c.over_estimate,label=labels[2])
axs[1, 0].plot(estimate_c.start_day,estimate_c.under_estimate,label=labels[3])
axs[1, 0].set_title('Group C')
axs[1, 0].legend(loc="upper left")

axs[1, 1].plot(estimate_nd.start_day,estimate_nd.over_estimate,label=labels[0])
axs[1, 1].plot(estimate_nd.start_day,estimate_nd.under_estimate,label=labels[1])
axs[1, 1].plot(estimate_d.start_day,estimate_d.over_estimate,label=labels[2])
axs[1, 1].plot(estimate_d.start_day,estimate_d.under_estimate,label=labels[3])
axs[1, 1].set_title('Group D')
axs[1, 1].legend(loc="upper left")


for ax in axs.flat:
    ax.set(xlabel='hour', ylabel='average estimate')

plt.tight_layout()

plt.show();